## Deliverable 2. Create a Customer Travel Destinations Map.

In [1]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps
# imports the previous directory for config.py
import sys
sys.path.insert(1, '../')

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [2]:
# 1. Import the WeatherPy_Database.csv file. 
city_data_df = pd.read_csv("../Weather_Database/WeatherPy_Database.csv")
city_data_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Vaini,TO,-21.20,-175.20,75.20,78,20,12.75,few clouds
1,1,San Pedro,PH,14.35,121.02,78.80,88,40,4.70,scattered clouds
2,2,Lagunas,PE,-5.23,-75.68,86.18,63,74,3.49,broken clouds
3,3,Washington,US,47.50,-120.50,30.99,92,40,6.93,scattered clouds
4,4,Luderitz,NaN,-26.65,15.16,73.40,56,0,24.16,clear sky


In [3]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 75
What is the maximum temperature you would like for your trip? 90


In [4]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head(10)

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Vaini,TO,-21.20,-175.20,75.20,78,20,12.75,few clouds
1,1,San Pedro,PH,14.35,121.02,78.80,88,40,4.70,scattered clouds
2,2,Lagunas,PE,-5.23,-75.68,86.18,63,74,3.49,broken clouds
11,11,Avarua,CK,-21.21,-159.78,78.80,94,96,8.05,moderate rain
15,15,Buala,SB,-8.14,159.59,78.84,78,100,5.86,light rain
18,18,Rikitea,PF,-23.12,-134.97,76.03,82,93,12.55,light rain
22,22,Lazaro Cardenas,MX,17.96,-102.20,78.64,72,0,3.29,clear sky
32,32,Georgetown,MY,5.41,100.34,82.40,88,20,3.36,light rain
35,35,Butaritari,KI,3.07,172.79,80.76,83,32,15.61,moderate rain
39,39,Saint-Philippe,RE,-21.36,55.77,78.80,69,75,5.82,broken clouds


In [5]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.count()

City_ID                178
City                   178
Country                177
Lat                    178
Lng                    178
Max Temp               178
Humidity               178
Cloudiness             178
Wind Speed             178
Current Description    178
dtype: int64

In [6]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
preferred_cities_df = preferred_cities_df.dropna()
preferred_cities_df.count()

City_ID                177
City                   177
Country                177
Lat                    177
Lng                    177
Max Temp               177
Humidity               177
Cloudiness             177
Wind Speed             177
Current Description    177
dtype: int64

In [7]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = preferred_cities_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
0,Vaini,TO,75.20,few clouds,-21.20,-175.20,
1,San Pedro,PH,78.80,scattered clouds,14.35,121.02,
2,Lagunas,PE,86.18,broken clouds,-5.23,-75.68,
11,Avarua,CK,78.80,moderate rain,-21.21,-159.78,
15,Buala,SB,78.84,light rain,-8.14,159.59,
18,Rikitea,PF,76.03,light rain,-23.12,-134.97,
22,Lazaro Cardenas,MX,78.64,clear sky,17.96,-102.20,
32,Georgetown,MY,82.40,light rain,5.41,100.34,
35,Butaritari,KI,80.76,moderate rain,3.07,172.79,
39,Saint-Philippe,RE,78.80,broken clouds,-21.36,55.77,


In [8]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    params["location"] = f"{lat},{lng}"    
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.")    
        

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


In [12]:
# 7. Drop the rows where there is no Hotel Name.
hotel_df.count()

              City Country  Max Temp Current Description    Lat     Lng  \
0            Vaini      TO     75.20          few clouds -21.20 -175.20   
1        San Pedro      PH     78.80    scattered clouds  14.35  121.02   
2          Lagunas      PE     86.18       broken clouds  -5.23  -75.68   
11          Avarua      CK     78.80       moderate rain -21.21 -159.78   
15           Buala      SB     78.84          light rain  -8.14  159.59   
..             ...     ...       ...                 ...    ...     ...   
649        Buluang      PH     76.82          few clouds  13.31  123.34   
650  Sao Sebastiao      BR     85.01           clear sky -23.76  -45.41   
653           Poum      NC     75.72           clear sky -20.23  164.02   
654       Itaituba      BR     84.20       broken clouds  -4.28  -55.98   
664         Vanimo      PG     79.03       moderate rain  -2.67  141.30   

                     Hotel Name  
0           Keleti Beach Resort  
1        OYO 412 Viewfort Venue

City                   177
Country                177
Max Temp               177
Current Description    177
Lat                    177
Lng                    177
Hotel Name             177
dtype: int64

In [13]:
hotel_df

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
0,Vaini,TO,75.20,few clouds,-21.20,-175.20,Keleti Beach Resort
1,San Pedro,PH,78.80,scattered clouds,14.35,121.02,OYO 412 Viewfort Venue
2,Lagunas,PE,86.18,broken clouds,-5.23,-75.68,Hospedaje Eco
11,Avarua,CK,78.80,moderate rain,-21.21,-159.78,Paradise Inn
15,Buala,SB,78.84,light rain,-8.14,159.59,Maringe Lagoon Lodge
...,...,...,...,...,...,...,...
649,Buluang,PH,76.82,few clouds,13.31,123.34,Enon Beach Resort
650,Sao Sebastiao,BR,85.01,clear sky,-23.76,-45.41,Hotel Recanto dos Pássaros
653,Poum,NC,75.72,clear sky,-20.23,164.02,Gîte kajeon
654,Itaituba,BR,84.20,broken clouds,-4.28,-55.98,Hotel Campos


In [15]:
# 8a. Create the output File (CSV)
output_data_file = "WeatherPy_vacation.csv"
# 8b. Export the City_Data into a csv
hotel_df.to_csv(output_data_file, index_label="City_ID")

In [17]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Current Weather</dt><dd>{Current Description}</dd>
<dt>Max Temp</dt><dd>{Max Temp} °F</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = hotel_df[["Lat", "Lng"]]

In [18]:
# 11a. Add a marker layer for each city to the map. 
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(marker_layer)

# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))